In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import csv

In [12]:
# Initialize blank.csv with header. Only needed when cars.csv doesn't exist.
# Separate from cars.csv so it doesn't get accidently overwritten.
# Rename to cars.csv if it doesn't exist.
columns = [
    "link",
    "listing_title",
    "listing_mileage",
    "primary_price",
    "deal_gauge",
    "exterior_color",
    "interior_color",
    "drivetrain",
    "mpg",
    "fuel_type",
    "transmission",
    "engine",
    "vin",
    "stock_number",
    "mileage",
    "entertainment",
    "exterior",
    "safety",
    "additional_popular_features",
    "accidents_or_damage",
    "1_owner_vehicle",
    "personal_use_only",
    "open_recall",
    "seller_name",
    "seller_rating",
    "seller_ratings_count",
    "sellers_notes",
    "consumer_rating_comfort",
    "consumer_rating_interior",
    "consumer_rating_performance",
    "consumer_rating_value",
    "consumer_rating_exterior",
    "consumer_rating_reliability",
    "consumer_ratings_count"
]

file_path = "blank.csv"
with open(file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(columns)

In [2]:
df = pd.read_csv("cars.csv")

In [ ]:
# Create a Selenium WebDriver
chrome_service = Service(executable_path='tools1_env/bin/chromedriver')
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
#chrome_options.add_argument("--headless") # Run without GUI
driver = webdriver.Chrome(options=chrome_options)

# Define the starting URL
listing_num = len(df) // 20 + 1
base_url = f"https://www.cars.com/shopping/results/?page_size=20&page={listing_num}&maximum_distance=10&sort=listed_at&zip=80210"
# Logic of code relies on: sort=listed_at ("Oldest listed") and page_size=20.
# maximum_distance and zip should be consistent for data integrity.
driver.get(base_url)

Webdriver starts on search page sorted by oldest first.
Dataframe reads from csv, then appends new data.
Duplicates will need to be removed after scraping.